# Exercício de Algoritmos de Classificação — Dataset de drogas/medicamentos
## Importação das bibliotecas e inicialização do dataframe

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("drug200.csv")
df

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [66]:
df.columns

Index(['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K', 'Drug'], dtype='object')

## Procurando por valores nulos ou Not a Number

In [67]:
df.isnull().sum()

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64

In [68]:
df.isna().sum()

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64

## Checando as drogas e a quantidade de cada uma

In [69]:
df['Drug'].value_counts()

Drug
DrugY    91
drugX    54
drugA    23
drugC    16
drugB    16
Name: count, dtype: int64

## Limpando as colunas e definindo o target da classificação

In [70]:
features = ['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']
target = 'Drug'

clean_df = df[features]
clean_target = df[target]

## Utilizando LabelEncoder para transformar as colunas de BP e Colesterol

In [71]:
from sklearn.preprocessing import LabelEncoder

le_bp = LabelEncoder()
le_col = LabelEncoder()

clean_df['BP'] = le_bp.fit_transform(clean_df['BP'])
clean_df['Cholesterol'] = le_col.fit_transform(clean_df['Cholesterol'])

C:\Users\disrct\AppData\Local\Temp\ipykernel_12476\2127780391.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['BP'] = le_bp.fit_transform(clean_df['BP'])
C:\Users\disrct\AppData\Local\Temp\ipykernel_12476\2127780391.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['Cholesterol'] = le_col.fit_transform(clean_df['Cholesterol'])


## Utilizando One-Hot Encoding para transformar a coluna de sexo

In [72]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

onehot_sex = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [1])], remainder='passthrough')
clean_df = onehot_sex.fit_transform(clean_df)

In [73]:
clean_df

array([[ 1.   ,  0.   , 23.   ,  0.   ,  0.   , 25.355],
       [ 0.   ,  1.   , 47.   ,  1.   ,  0.   , 13.093],
       [ 0.   ,  1.   , 47.   ,  1.   ,  0.   , 10.114],
       ...,
       [ 0.   ,  1.   , 52.   ,  2.   ,  0.   ,  9.894],
       [ 0.   ,  1.   , 23.   ,  2.   ,  1.   , 14.02 ],
       [ 1.   ,  0.   , 40.   ,  1.   ,  1.   , 11.349]], shape=(200, 6))

## Colunas transformadas (anotações)

clean_df[0] = Feminino
clean_df[1] = Masculino
clean_df[3] = BP — 0 = HIGH, 1 = LOW, 2 = NORMAL
clean_df[4] = Cholesterol — 0 = HIGH, 1 = NORMAL

## Separando dados de treinamento e dados de teste

In [74]:
from sklearn.model_selection import train_test_split

x_drugs_train, x_drugs_test, y_drugs_train, y_drugs_test = train_test_split(clean_df, clean_target, test_size=0.3, random_state=0)
x_drugs_train.shape, x_drugs_test.shape

((140, 6), (60, 6))

# Testando diferentes algoritmos de classificação
## Naive Bayes

In [75]:
from sklearn.naive_bayes import GaussianNB

naive_drugs = GaussianNB()
naive_drugs.fit(x_drugs_train, y_drugs_train)
predictions = naive_drugs.predict(x_drugs_test)
predictions

array(['drugC', 'drugX', 'DrugY', 'DrugY', 'DrugY', 'drugX', 'drugX',
       'drugX', 'drugC', 'drugX', 'drugA', 'DrugY', 'DrugY', 'DrugY',
       'drugB', 'drugC', 'DrugY', 'DrugY', 'drugX', 'drugA', 'drugA',
       'drugX', 'drugX', 'drugX', 'DrugY', 'DrugY', 'DrugY', 'DrugY',
       'DrugY', 'drugX', 'drugX', 'drugC', 'drugA', 'drugX', 'DrugY',
       'DrugY', 'drugX', 'drugC', 'drugA', 'drugX', 'DrugY', 'drugA',
       'DrugY', 'DrugY', 'DrugY', 'drugX', 'drugX', 'DrugY', 'drugA',
       'drugB', 'DrugY', 'drugB', 'DrugY', 'DrugY', 'drugC', 'DrugY',
       'drugX', 'drugX', 'DrugY', 'drugB'], dtype='<U5')

In [76]:
np.array(y_drugs_test)

array(['drugC', 'drugX', 'DrugY', 'DrugY', 'DrugY', 'drugX', 'drugX',
       'drugX', 'DrugY', 'drugX', 'drugA', 'DrugY', 'DrugY', 'DrugY',
       'drugB', 'drugC', 'DrugY', 'DrugY', 'drugX', 'DrugY', 'DrugY',
       'drugX', 'drugX', 'drugX', 'DrugY', 'DrugY', 'DrugY', 'DrugY',
       'DrugY', 'drugX', 'drugX', 'drugC', 'drugA', 'drugX', 'DrugY',
       'DrugY', 'drugX', 'DrugY', 'drugA', 'drugX', 'DrugY', 'drugA',
       'DrugY', 'drugX', 'DrugY', 'drugX', 'DrugY', 'DrugY', 'drugA',
       'DrugY', 'DrugY', 'drugB', 'DrugY', 'DrugY', 'drugC', 'DrugY',
       'drugX', 'drugX', 'DrugY', 'drugB'], dtype=object)